In [83]:
from dotenv import load_dotenv,find_dotenv
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI
from langchain_google_genai import GoogleGenerativeAI,ChatGoogleGenerativeAI
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [10]:
load_dotenv(find_dotenv("D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [11]:
db=SQLDatabase.from_uri(database_uri="sqlite:///db/chinook.db/chinook.db")

In [12]:
db.dialect

'sqlite'

In [13]:
db.get_usable_table_names()

['Album',
 'Artist',
 'Customer',
 'Employee',
 'Genre',
 'Invoice',
 'InvoiceLine',
 'MediaType',
 'Playlist',
 'PlaylistTrack',
 'Track']

In [14]:
sql="""
    SELECT
        *
    FROM
        artist
    LIMIT
        10;
"""
db.run(command=sql)

"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [74]:
# llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash-001",temperature=0.3)
llm=ChatOpenAI(model="gpt-3.5-turbo")

In [75]:
chain=create_sql_query_chain(llm=llm,db=db)

In [76]:
query={"question":"How many Employees are there? Only provide the query. Provide the response without a key-value pair"}

response=chain.invoke(input=query)

In [77]:
response.strip()

'SELECT COUNT(*) FROM Employee;'

In [78]:
db.run(command=response.split(":")[-1])

'[(8,)]'

In [79]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

### Using QuerySQLDataBaseTool

In [80]:
executeQuery=QuerySQLDataBaseTool(db=db)

In [81]:
writeQuery=create_sql_query_chain(llm=llm,db=db)
chain=writeQuery|executeQuery

In [82]:
chain.invoke(input=query)

'[(8,)]'

### Using Question Answering

In [88]:
chain=RunnablePassthrough.assign(query=writeQuery).assign(result=itemgetter("query"))|executeQuery
chain.invoke(input=query)

'[(8,)]'

In [90]:
chain=RunnablePassthrough.assign(query=writeQuery).assign(result=itemgetter("query"))|executeQuery
chain.invoke(input=query)

'[(8,)]'

In [132]:
answerPrompt=PromptTemplate.from_template(
    template="""
        Given the following user question, corresponding SQL query and SQL Result,
        answer the user's question:

        Question: {question}
        SQL Query: {query}
        SQL Result: {result}
        Answer:
    """
)


answer=answerPrompt | llm | StrOutputParser()

In [133]:
chain = (RunnablePassthrough.assign(query=writeQuery).assign(result=itemgetter("query") | executeQuery) | answer)

In [135]:
chain.invoke(query)

'There are 8 Employees.'